 ### test service
 Работа Севриса: 
 - Загруpка данных с PostresSQL (либо загрузка файлов локальо - с csv)
 - Загрузка модели
 - Предсказание постов для конкретного user_id
 - Возвращение ответа

In [5]:
import pandas as pd
from sqlalchemy import create_engine

 **1) Загруpка данных с PostresSQL (либо загрузка файлов локальо - с csv)**
 * Загрузка данные батчами
 * загружаем признаки отдельно по юзерам, отдельно по постам 

In [8]:
password = '*****' #При необходимоси пароль может быть скинут

In [7]:
SQLALCHEMY_DATABASE_URL = f"postgresql://robot-startml-ro:{password}@" \
                          "postgres.lab.karpov.courses:6432/startml"
engine = create_engine(SQLALCHEMY_DATABASE_URL)

In [9]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 500
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn,
                                       chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
        print(chunk_dataframe)
    conn.close()
    data = pd.concat(chunks, ignore_index=True)

    return data

In [11]:
def load_features() -> pd.DataFrame:
    query = "v_patrakeev_all_posts"
    query2 = "v_patrakeev_all_users"
    return batch_load_sql(query), batch_load_sql(query2)

In [13]:
#posts, users = load_features()
posts, users = pd.read_csv("data csv/post_feature_to_SQL_new", sep=";"),pd.read_csv("data csv/user_feature_to_SQL_new", sep=";")

In [14]:
posts.head()

,post_id,Topic,text_length,likes,posts_views,likes_share
0,7319,10.733667,790,720.0,6585.0,10.0
1,7318,10.733667,728,680.0,6785.0,10.0
2,7317,10.733667,636,731.0,6803.0,10.0
3,7316,10.733667,800,677.0,6572.0,10.0
4,7315,10.733667,803,2619.0,16940.0,15.0


In [15]:
users.head()

,user_id,age,City,business,covid,entertainment,movie,politics,sport,tech,views,ratio of likes posts
0,168552,16,0.101590,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
1,168551,38,0.159781,0.138777,0.250000,0.400000,0.125000,0.000000,0.000000,0.000000,30.0,0.166667
2,168550,41,0.107315,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
3,168549,18,0.109604,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,34.0,0.029412
4,168548,36,0.099558,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435


 **2) Исходные данные**
 * Вводим входные данные: юзер ( для которого будет предсказание, время запроса, лимит постов для рекомендаций) 

In [37]:
from datetime import datetime
id = 10001
time = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #time recome
limit = 5

 **3) Список вcех постов, которые смотрел пользователь.**
 - Запрос в базу данных со всеми активностями (где у нас более 80 млн строr, быстро через SQL)

In [38]:
user_action_view = list(pd.read_sql(
     f"""
     select *
     from public.feed_data
     where user_id = {id} AND action = 'view'
         ;""",
     con=SQLALCHEMY_DATABASE_URL
 )['post_id'])

 **4) Список вcех постов, что у нас вообще есть**

In [40]:
all_posts = list(pd.read_sql(
     f"""
     select id
     from public.posts
         ;""",
     con= SQLALCHEMY_DATABASE_URL
 )['id'])

 **5) Список постов, что пользователь не смотрел и мы ему можем их рекомендовать**

In [42]:
spisok_posts = list(set(all_posts) - set(user_action_view))

 **6) Дата фрейм (каркас) для дополнения его признакми по пользователю постам и вероятностями того или иного поста**

In [43]:
df = pd.DataFrame({'user_id': id,
                   'post_id': spisok_posts,
                   'time': time})

In [44]:
#7 add our features from download early
df = df.merge(posts,how='left',left_on='post_id', right_on='post_id')
df = df.merge(users,how='left',left_on='user_id', right_on='user_id')
df.head()

,user_id,post_id,time,Topic,text_length,likes,posts_views,likes_share,age,City,business,covid,entertainment,movie,politics,sport,tech,views,ratio of likes posts
0,10001,1,2023-10-26 13:19:18,10.513725,1967,1067.0,8508.0,12.0,26,0.141995,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
1,10001,2,2023-10-26 13:19:18,10.513725,2701,637.0,7495.0,8.0,26,0.141995,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
2,10001,3,2023-10-26 13:19:18,10.513725,3408,1122.0,8412.0,13.0,26,0.141995,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
3,10001,4,2023-10-26 13:19:18,10.513725,1026,1171.0,8158.0,14.0,26,0.141995,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
4,10001,5,2023-10-26 13:19:18,10.513725,889,1153.0,8583.0,13.0,26,0.141995,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435


 **7) Загрука моделей**

In [51]:
#8 download our save model
from catboost import CatBoostClassifier

model = CatBoostClassifier()
model = model.load_model('Models/model_second_catboost', format="cbm")

 **8) Предсказание**

In [57]:
pred = pd.DataFrame(model.predict_proba(df.drop(columns=['user_id','post_id', 'time']))[:, 1], columns=["predict"])

 **9) Добавление предсказаний к каждому посту, который не смотрел пользователь**

In [54]:
#add our
df = pd.concat([df, pred], axis=1)

 **10) Сортировка**

In [60]:
df = df.sort_values(by=["predict"], ascending=False)

**11) Вывод 5 посстов ( их ID)**

In [61]:
spisok = tuple(df["post_id"].head(limit).tolist())
spisok

(2013, 2070, 2148, 2087, 2166)

**12) Вывод датафрейма с постами и их описанием**

In [62]:
post_top5_spisok = pd.read_sql(
    f"""
     select *
     from public.post
     where id IN {spisok}

         ;""",
     con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml"
 )


In [63]:
post_top5_spisok

,id,text,topic
0,2013,Call for action on internet scam\n\nPhone comp...,tech
1,2070,Who do you think you are?\n\nThe real danger i...,tech
2,2087,Podcasters look to net money\n\nNasa is doing ...,tech
3,2148,No half measures with Half-Life 2\n\nCould Hal...,tech
4,2166,Football Manager scores big time\n\nFor the pa...,tech
